In [1]:
def read_csv_list(my_tag):
    tag_name = my_tag
    file_name = tag_name + '_tag_link.csv'
    link_list = []
    
    with open(file_name) as file:
        for line in file:
            read_data = file.readline()
            link_list.append(read_data[:-2])
        file.close()
    return link_list

my_tag = 'seoul'
seoul_links = read_csv_list(my_tag)
print(seoul_links[0])
print(len(seoul_links))

https://www.instagram.com/p/BsXepyGBjHr/
82


In [2]:
## 추출하고자 하는 내용은 다음과 같습니다.

## id ( 게시글 아이디 ) -> OK
## username ( 유저 아이디) -> OK
## date ( 게시 날짜 ) -> OK
## contents ( 글 내용 ) -> OK
## tags ( 태그들 ) -> OK
## image_links ( 이미지 링크들 ) -> OK
## like_num ( 좋아요 개수 ) -> OK
## comment_num ( 댓글 개수 ) -> OK

In [3]:
import requests
import re
import time
import json
from bs4 import BeautifulSoup

In [4]:
## 찾아낸 데이터를 JSON 객체로 생성하는 함수를 정의합니다.
def data2json(my_tag, id, username, date, contents, hash_tags, final_image_link, likes_num, comments_num):
    single_data = {
        "find_tag" : my_tag,
        "id" : id,
        "username" : username,
        "date" : date,
        "contents" : contents,
        "hashtags" : hash_tags,
        "imagelinks" : final_image_link,
        "likes" : likes_num,
        "comments" : comments_num
    }
    return single_data

In [8]:
json_list = []

for i in range(len(seoul_links)):
    
    test_url = seoul_links[i]
    req = requests.get(test_url)
    html = req.text
    header = req.headers
    status = req.status_code
    soup = BeautifulSoup(html, 'html.parser')
    script_content = soup.find_all('script')

    ## 게시글 아이디를 링크로부터 가져와 찾아냅니다.
    id_p = re.compile("\/p\/(.*?)\/")
    id = id_p.findall(test_url)[0]
    print(id)

    ## 사용자 이름을 rel='canonical'에서 주소를 가져와 찾아냅니다.
    user_content = soup.find(rel="canonical")
    if user_content:
        username_pp = user_content.get('href')
        username_p = re.compile("instagram.com/(.*)/p")
        username = username_p.findall(str(username_pp))
        if len(username) > 0:
            username = username[0]
        print(username)


    ## 날짜를 header 파일에서 'Date' 부분을 가져와 찾아냅니다.
    date_p = re.compile("uploadDate\":\"(.{19})\"")
    date = date_p.findall(str(script_content))
    if len(date) > 0:
        date = date[0]
    else:
        date = 0
    print(date)

    ## 글 내용을 script 내용에서 caption 부분을 가져와 찾아냅니다.
    ## 태그 내용을 contents 에서 빼는 과정이 필요할지도 모릅니다.
    if script_content:   
        contents_p = re.compile("\"caption\":\"(.*?)\"")
        contents = contents_p.findall(str(script_content))
        if len(contents) > 0:
            contents = contents[0]
        print(contents)

    ## 해쉬태그를 property='instapp:hashtags' 에서 contetn= 이후 부분을 가져와 찾아냅니다.
    meta_content = soup.find_all(property = "instapp:hashtags")
    if meta_content:
        hash_tags_p = re.compile("content=\"(.*?)\"")
        hash_tags = hash_tags_p.findall(str(meta_content))
        print('해쉬태그 개수 : ', len(hash_tags))
        print(hash_tags)

    ## 이미지 링크를 display_resources 에서 가져와 찾아냅니다.
    ## 이미지가 N개일 때, 총 N+1 종류의 이미지 링크가 존재합니다.
    ## 확인해 보니 1번째와 2번째 이미지는 동일하며, 나머지는 서로 다른 이미지였습니다.
    ## 추정해 보면 1번째 이미지는 2번째 이후 이미지들을 대표하는 하나의 이미지라 할 수 있습니다.
    ## 각 이미지에 대해 서로 다른 resolution 으로 3가지의 다른 형태로 저장된 것을 볼 수 있습니다.
    ## 1번째는 640, 2번째는 750, 3번째는 원본 크기인것으로 보이며, 이 중에서 원본 크기의 링크만 저장합니다.
    image_links_p = re.compile("\"display_resources\":\[.*?\]")
    image_links = image_links_p.findall(str(script_content))

    final_image_link = []
    single_image_link_p = re.compile("\"src\":\"(.*?)\"")
    for dif_img in image_links:
        link = single_image_link_p.findall(str(dif_img))[2]
        final_image_link.append(link)

    ## 이미지가 1개일 때는 대표 이미지만 존재하므로, 이에 대한 예외처리가 필요하다
    if len(final_image_link) != 1:
        final_image_link = final_image_link[1:]
    print('이미지 개수 : ', len(final_image_link))
    for i in range(len(final_image_link)):
        print(final_image_link[i])

    ## like_num, comment_num 을 description 에서 찾아냅니다.
    likes_num_p = re.compile("\"description\":\"(.*?)Likes")
    likes_num = likes_num_p.findall(str(script_content))
    if len(likes_num) > 0:
        likes_num = likes_num[0]
    else:
        likes_num = 0
    print(likes_num)
    comments_num_p = re.compile("\"description\":\".*?,(.*?)Comments")
    comments_num = comments_num_p.findall(str(script_content))
    if len(comments_num) > 0:
        comments_num = comments_num[0]
    else:
        comments_num = 0
    print(comments_num)
    
    time.sleep(0.1)
    
    print('------------------')
    single_json = data2json(my_tag, id, username, date, contents, hash_tags, final_image_link, likes_num, comments_num)
    json_list.append(single_json)


BsXepyGBjHr
luciefer_stay
2019-01-08T08:27:58
#seoul #southkorea #coreedusud #travel #bongeunsa #templebouddhiste #temple #serenite #magnifique
해쉬태그 개수 :  9
['seoul', 'travel', 'bongeunsa', 'temple', 'serenite', 'coreedusud', 'southkorea', 'magnifique', 'templebouddhiste']
이미지 개수 :  1
https://scontent-icn1-1.cdninstagram.com/vp/00d514738585eb1bd1e908a23a6f71e2/5CB34BAA/t51.2885-15/e35/50073119_283646015841060_1569874452121957529_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
21 
 0 
------------------
BsXfiLXhDIO
0
[]
이미지 개수 :  0
0
0
------------------
BsXezP3hPrl
pic.724
0
[]
해쉬태그 개수 :  30
['picoftheday', '서울', '감성', 'mood', 'instatravel', '코코카피탄', '데일리', 'travelgram', 'f4f', '여행스타그램', '일상', 'seoul', '전시', '여행에미치다', 'travelphotography', '대림미술관', 'travel', '글', 'iphone', 'somewheremagazine', '사진전', 'daily', 'cococapitan', 'selfie', 'landscape', '여행', 'art', '사진', 'snap', 'photography']
이미지 개수 :  1
https://scontent-icn1-1.cdninstagram.com/vp/7c516b2e5fb6c578cc55264c84b3ab7e/5CCF1B5F/t51.

BsXejsoFvGV
lalalaliza
0
[]
이미지 개수 :  1
https://scontent-icn1-1.cdninstagram.com/vp/229dce6ded56d3a7bf77c3ce89fc19d0/5CC662F1/t51.2885-15/e35/p1080x1080/47586025_2234033470218319_8567868035479825737_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
0
0
------------------
BsXev3ml35h
minto.meow
2019-01-08T08:28:48
\u2800\n\uc5b4\ub5a4 \uc5ec\uc790\ub4e4\uc740 \uc218\ud37c\ub9e8\uc744 \uc6d0\ud558\uc9c0\ub9cc, \ub9e4\uc77c \ud074\ub77c\ud06c \ucf04\ud2b8\ub97c \ubcf4\uba74\uc11c\ub3c4 \uc9c0\ub098\uccd0 \ubc84\ub9b0\ub2e4.\n\u2800\n\u041d\u0435\u043a\u043e\u0442\u043e\u0440\u044b\u0435 \u0434\u0435\u0432\u0443\u0448\u043a\u0438 \u0445\u043e\u0442\u044f\u0442 \u0441\u0443\u043f\u0435\u0440\u043c\u0435\u043d\u0430, \u043d\u043e \u043f\u0440\u0438 \u044d\u0442\u043e\u043c \u043a\u0430\u0436\u0434\u044b\u0439 \u0434\u0435\u043d\u044c \u043f\u0440\u043e\u0445\u043e\u0434\u044f\u0442 \u043c\u0438\u043c\u043e \u041a\u043b\u0430\u0440\u043a\u0430 \u041a\u0435\u043d\u0442\u0430. \u2800\n#memories #ru

BsXfFK3gd7M
slpsiusin
2019-01-08T08:31:42
People...\n.\n.\n.\n#streets_storytelling #ig_streetclub\n#ig_photo_club #hikaricreative #seoul #capturestreets\u00a0#streetleaks #challengerstreets #storyofthestreet #lensculturestreets #everybodystreet #street_photographer  #xplorethestreets #streetlife_spi #streetmagazine #streetphography #lensonstreets #streets_unseen #magnumphotos #380collective  #yallersstreet #apfmagazine #life_is_street #photo_collective #gettyimages #fromstreetswithlove #streetizm #balkancollective #streetsgrammer  #quietstranger
해쉬태그 개수 :  30
['everybodystreet', 'ig_streetclub', 'life_is_street', 'streetsgrammer', 'street_photographer', 'streets_unseen', 'quietstranger', 'xplorethestreets', '380collective', 'challengerstreets', 'streetlife_spi', 'gettyimages', 'storyofthestreet', 'seoul', 'balkancollective', 'apfmagazine', 'hikaricreative', 'streetleaks', 'magnumphotos', 'yallersstreet', 'lensonstreets', 'streets_storytelling', 'lensculturestreets', 'ig_photo_club', '

BsXe8U_nwfv
harixtrash
0
[]
해쉬태그 개수 :  8
['korea', 'seoul', '서울', 'random', '데일리', '한국', 'southkorea', 'life']
이미지 개수 :  10
https://scontent-icn1-1.cdninstagram.com/vp/488b1e32e4555020404ea7e5ad8e7775/5CBEB89F/t51.2885-15/e35/47585128_294761978058777_7902727553067736064_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
https://scontent-icn1-1.cdninstagram.com/vp/dfb75c367d7bfa4e44d365f0a1bbe8cf/5CD8031D/t51.2885-15/e35/47581852_2407436249298210_5391913425440871818_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
https://scontent-icn1-1.cdninstagram.com/vp/2f631c4d4e62a9a1c77b3cf8ce2cd516/5CBA4D12/t51.2885-15/e35/47584685_323470065176278_5793214437909897652_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
https://scontent-icn1-1.cdninstagram.com/vp/480777b550516888284210baef82e2e2/5CCE1A96/t51.2885-15/e35/47585174_492622667934153_5880166130114637164_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
https://scontent-icn1-1.cdninstagram.com/vp/1329ae0d450b045661c4dc4a3325af55/5CC24030/t51.2885-15/e35/4

BsXea9HBu3A
scrys1
2019-01-08T08:25:57
#korea #seoul #cafe
해쉬태그 개수 :  3
['cafe', 'korea', 'seoul']
이미지 개수 :  1
https://scontent-icn1-1.cdninstagram.com/vp/facae61cbf84940e7a19416ce14e1d4c/5CD623C2/t51.2885-15/e35/47586218_681054792291032_1784985608383346279_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
0 
 1 
------------------
BsXgEUYBhb7
amandarikim
2019-01-08T08:40:20
.\nMy favourite peoples as usual.\n.\n.\n#friends #seoul #welcome
해쉬태그 개수 :  3
['welcome', 'seoul', 'friends']
이미지 개수 :  1
https://scontent-icn1-1.cdninstagram.com/vp/fbd8060eb3ff78ef446bda7b54596aa6/5CC840EF/t51.2885-15/e35/47586540_348893642596599_2206198722088583435_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
43 
 1 
------------------
BsXfGYKBfuY
link_u_korea
2019-01-08T08:31:52
Instagram starts! \ud83c\udf89\nI&#x27;ll let you know a lot about Korea! \nPlease ask me anything! \ud83d\udc40\u263a\ufe0f
이미지 개수 :  1
https://scontent-icn1-1.cdninstagram.com/vp/25968f4d4318aac76748493c2a4b65d5/5CB2A201/t51.2885-15/e35

...

BsXem28HyIj
maryjane__kd
2019-01-08T08:27:34
Nella natura\ud83c\udf42\ud83c\udf32\u2022
이미지 개수 :  1
https://scontent-icn1-1.cdninstagram.com/vp/727d60c11e3c8e3ffbcc3dccbc7a5537/5CC3B5B5/t51.2885-15/e35/49459398_309252343056982_593375617756176662_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
99 
 4 
------------------
BsXezl4F2gF
kai0409
2019-01-08T08:29:18
*\n2019\/1\n*\n*\n\u65c5\u306e\u8a18\u9332\ud83d\udcf8\n*\n\u97d3\u56fd\u3067\u6b32\u3057\u304b\u3063\u305f\u3053\u306e\u30d5\u30e9\u30a4\u8fd4\u3057\u3002\n\u50ac\u4e8b\u3058\u3083\u306a\u3044\u3068\u58f2\u3063\u3066\u306a\u3044\u3060\u308d\u3046\u3068\u601d\u3063\u3066\u554f\u3044\u5408\u308f\u305b\u305f\u3089\u307e\u3055\u304b\u306e\u2026\u53d7\u6ce8\u8ca9\u58f2\u2190\u307e\u305f\ud83d\ude05\ud83d\ude05\ud83d\ude05\n*\n\u6ce8\u6587\u304b\u30896\u9031\u9593\u3067\u51fa\u6765\u4e0a\u304c\u308b\u3063\u3066\ud83d\ude2d\n\u6ede\u5728\u306f10\u65e5\u9593\ud83d\udca6\n\u307e\u305f\u3082\u3084\u3063\u3061\u307e\u3063\u305f\u611f\u6e80\u8f

BsXfVhUjpmg
gf15exo
2019-01-08T08:33:56
EXO PLANET 5 TOUR IN SEOUL\nVANUE: OLYMPIC GYMNASTIC ARENA\nDATES: FEB 15-17, 22-24 (6 days)\n\n#\uc5d1\uc18c #EXO @weareone.exo #\uc5ec\uc790\uce5c\uad6c #GFRIEND @gfriendofficial #EXOPlanet5 #seoul
해쉬태그 개수 :  6
['gfriend', 'seoul', 'exoplanet5', '엑소', '여자친구', 'exo']
이미지 개수 :  1
https://scontent-icn1-1.cdninstagram.com/vp/a3caa0875efd1d71401a562876bd458e/5CB3A689/t51.2885-15/e35/49737009_510017052820285_2210062211790241135_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
6 
 0 
------------------
BsXedYwhiFW
y1k9
2019-01-08T08:26:17
\ucfe0\ud0a4\u2763\ufe0f
해쉬태그 개수 :  26
['스탬프커피', '延南洞カフェ', '韓国カフェ', '연남동카페', '韓国', '카페투어', '韓国旅行', '延南洞', '심플라이프', 'カフェ巡り', '카페스타그램', 'クッキー', '연남동핫플', '연남동', 'インテリア', '핫플레이스', 'ソウル', 'seoulcafe', 'ヨンナムドン', 'ソウルカフェ', 'ソウル旅行', '韓国情報', 'stampcoffee', 'cookies', '쿠키', 'seoul']
이미지 개수 :  1
https://scontent-icn1-1.cdninstagram.com/vp/a2c49d9e61af65a61517d86e6a53ef86/5CD55A6A/t51.2885-15/e35/47693876_323093954971517_1713379662

BsXf7O7A-90
___kim_taetae____
2019-01-08T08:39:05
19\/01\/06 Golden Disc Awards 2019\n#\uae40\ud0dc\ud615#BTS#taetae#suga#minyoongi#bangtanboys#bangtansonyeondan#airplane#army#fakelove#followme#f4f#kimtaehyung#taehyung#korean#blackpink#exo#got7#loveyourself#bt21#seoul#jin#jimin#jungkook#rapmonster#jhope#V#likeforfollow#likeforlike#vkook
해쉬태그 개수 :  30
['bangtansonyeondan', 'taetae', 'jin', 'blackpink', 'bts', 'seoul', 'jimin', 'v', 'airplane', 'bangtanboys', 'f4f', 'fakelove', 'jhope', 'minyoongi', 'taehyung', 'likeforfollow', '김태형', 'bt21', 'followme', 'likeforlike', 'loveyourself', 'suga', 'exo', 'korean', 'rapmonster', 'kimtaehyung', 'vkook', 'got7', 'army', 'jungkook']
이미지 개수 :  4
https://scontent-icn1-1.cdninstagram.com/vp/b383f71cdfb5fe265033bed9aead6b22/5CD994BE/t51.2885-15/e35/49332803_2337295286557057_420752174844653472_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
https://scontent-icn1-1.cdninstagram.com/vp/e0973a2e99263e0018f761bd9bc5e4ac/5CB3EEAC/t51.2885-15/e35/47692877_153

BsXefxHl7uX
hodoo__papa
2019-01-08T08:32:48
\ub610 \ud574\ub2ec\ub77c\uad6c?\/\u307e\u305f\u3057\u3066\u304f\u308c\u3063\u3066?\/Do it again?\ud83d\ude00\n.\uacc4\uc18d \ub9cc\uc838\uc57c.~^^\/\u624b\u3067\u3064\u3065\u3051\u306a\u3044\u3068...\/He like to touch\ud83d\ude1b\n.\n.\n.\n.\n\n#\uc560\uacac#\uac15\uc544\uc9c0#\uba4d\uc2a4\ud0c0\uadf8\ub7a8#\uc720\uba38#\ub315\ub315\uc774#\ub610#\uc11c\uc6b8#\ud478\ub4e4#\ub178\ud2b89 #Puppy#humor#daily#dogs#note9#again#dogstagram#korea#pet#seoul#toypoodle#cute#\u30e9\u30a4\u30d5\uff03\u30b9\u30de\u30fc\u30c8\u30d5\u30a9\u30f3#\u307e\u305f#\u72ac\uff03\u97d3\u56fd\uff03\u304b\u308f\u3044\u3044\uff03\u30c8\u30a4\u30d7\u30fc\u30c9\u30eb\uff03\u3044\u306c#\u30bd\u30a6\u30eb
이미지 개수 :  1
https://scontent-icn1-1.cdninstagram.com/vp/cca851c797dba4f8a1d22686e241bbe9/5C3860F0/t51.2885-15/e35/47689966_2293499757561758_685907207740635107_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
246 
 4 
------------------
BsXgUNeBPIR
youngsusoo
2019-01-08T08:42:30

BsXfbmFhNoI
0
[]
이미지 개수 :  0
0
0
------------------
BsXgQTGFzFK
kwondo_photo
2019-01-08T08:41:58
Bakery in Seoul\n\u30bd\u30a6\u30eb\u306e\u30d1\u30f3\u5c4b\u3055\u3093\n\n#iphone #iphonephotography #iphonephoto #korea #morning #tbt #\u97d3\u56fd #\u97e9\u56fd #\u9996\u5c14 #Seoul #photooftheday #picoftheday #bakery #theweekofinstagram #instagood #pt_life_ #instagram #daily #likes
해쉬태그 개수 :  19
['seoul', 'theweekofinstagram', 'instagood', 'iphonephoto', 'picoftheday', 'iphone', 'pt_life_', 'photooftheday', '韓国', '韩国', 'iphonephotography', 'korea', 'instagram', 'daily', 'morning', 'tbt', 'likes', 'bakery', '首尔']
이미지 개수 :  1
https://scontent-icn1-1.cdninstagram.com/vp/803d15840ba59c9c292bf371f56b83c6/5CD23E0C/t51.2885-15/e35/47689994_300626020592421_729712486669967679_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
18 
 0 
------------------
BsXfiqIh0Mu
nuts_andboltss
2019-01-08T08:35:44
-\nWINTER | JANUARY | NEW ARRIVAL\n\u25aa\ufe0e\u77ed\u8eab\u885b\u8863 3\u500b\u8272 \u73fe\u8ca8\u5f9

In [9]:
len(json_list)

82

In [10]:
## 생성한 데이터를 파일 형태로 저장합니다.

def save_jsonlist_json(myjson):
    file_name = my_tag + '_rawdata.json'
    with open(file_name, "w") as file:
        for line in myjson:
            file.write(str(line) + ',\n')

save_jsonlist_json(json_list)

In [11]:
## 앞으로 해야할 일

## 1. Like 와 Comment 형태를 1,000 등의 string 에서 1000 의 int 로 변환 저장
## 2. Unicode 형태의 데이터를 읽을 수 있는 데이터로 변환 -> 이모티콘을 출력하려 할 경우 멈춤
##    이모티콘을 제외하고 변환하여 사용하는 것이 맞을 것으로 보임
##    이와 관련해서는 ignore 옵션을 사용할 수 있을 것으로 보임